# 20181113 Ratiometric comparisons to resolve cross-talking signals

## Goal
* develop theory and figures to show that ratiometric comparison of pairs of cross-talking sensors can be used to resolve which chemical signal is present
* hope to find a large concentration-invariant regime
* Largely inspired by: 10.1093/protein/gzi069

### System
* 3 separate chemical ligands that differentially activate receptors
* 2 receptors (sensors) with differential activation
* each signal is presented individually to the system

## Approach
* simulate inducer-dependent sensor response with activating Hill functions
* 6 different Hill functions (2 for each inducer, one for each receptor)
* look at ratios of sensor activation over full induction range with each

## Questions / things to think about
* what kind of system would be needed to constructed to test for multiple, simultaneous ligand types
    * might be good to look at the follow up paper: https://doi.org/10.1371/journal.pcbi.1002224
* what do the activation functions of cross-talking receptors usually look like
    * Adam says they usually have different KD values
    
## Example cross-talk from Adam's QS systems
* Plots along the diagonal line are cognate activator:promoter pairs.  
* other plots are non-cognate promoters.  Any non-flat transfer functions in those plots represent promoter cross reactivity.
* each different color of transfer function is for a different ligand.  
* two non-flat transfer functions in the same plot represent chemical cross-reactivity
<img src="QS_crosstalk.png">

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('ticks')
sns.set_context('poster')

In [ ]:
from IPython.core.debugger import set_trace

In [ ]:
# Create Hill function, function; takes vector of inputs, returns vector of outputs
def hill(x_vals, y_min = 0.005, y_max = 1, kd = 1, n = 2):
    outputs = []
    for x in x_vals:
        out =  y_min+((y_max-y_min)/(1+(kd/x)**n))
        outputs.append(out)
    return np.array(outputs)

## Plotting examples

In [ ]:
# Practice plotting one hill function
xs = np.logspace(-2,1,100)
ys = hill(xs)

fig, ax = plt.subplots()
ax.plot(xs,ys)
ax.set_xscale('log')
ax.set_yscale('log')
plt.show()

In [ ]:
# Plot two with just different ymaxes
xs = np.logspace(-2,1,100)
y1s = hill(xs)
y2s = hill(xs, y_max = 0.1)

fig, ax = plt.subplots()
ax.plot(xs,y1s)
ax.plot(xs,y2s)
ax.set_xscale('log')
ax.set_yscale('log')
plt.show()

In [ ]:
# Plot two with different kds and ymaxs
xs = np.logspace(-2,1,100)
y1s = hill(xs)
y2s = hill(xs, kd = 5, y_max = 0.1)

fig, ax = plt.subplots()
ax.plot(xs,y1s)
ax.plot(xs,y2s)
ax.set_xscale('log')
ax.set_yscale('log')
plt.show()

### Results
* at least 10-fold separation from inputs of $10^{-1}$ and up
* this does NOT yeild new information since we could just look at P2 and see that we get more than 10-fold separation if input is more than $10^{-1}$

# Piping through simulated ratiometric circuit

Define the ratiometric circuit calculator function

In [ ]:
def ratio_circ(inputs_1, inputs_2, kd_kr_ratio = 1, n =0.7):
    
    kr = 0.001/30/(2.9*10**(-3))
    kd = kd_kr_ratio*kr
    scale = 4
    
    pairs = list(zip(inputs_1,inputs_2))
    
    output = []
    for pair in pairs:
        out = scale*(1/(1+pair[0]**n/kd + pair[1]**n/kr) + 1/(1+kd/pair[0]**n + pair[1]**n*kd/(pair[0]**n*kr)))
        output.append(out)
    return np.array(output)

In [ ]:
# Makes pairs of inputs from sets
def pair_maker(inputs_1,inputs_2):
    pairs = []
    for i in inputs_1:
        for j in inputs_2:
            pairs.append((i,j))
    return pairs

inputs = np.logspace(-2,1,50)*100
pairs = np.array(pair_maker(inputs,inputs))
real_ratios = pairs[:,0]/pairs[:,1]

outputs_same_k = ratio_circ(pairs[:,0],pairs[:,1], kd_kr_ratio=1)
outputs_high_k = ratio_circ(pairs[:,0],pairs[:,1], kd_kr_ratio=10)
outputs_v_high_k = ratio_circ(pairs[:,0],pairs[:,1], kd_kr_ratio=100)

In [ ]:
fig, ax = plt.subplots(1,3,sharey=True)

ax[0].scatter(real_ratios,outputs_same_k,s =20)
ax[0].set_title('Kd = Kr')

ax[1].scatter(real_ratios,outputs_high_k,s =20)
ax[1].set_title('Kd = 10*Kr')

ax[2].scatter(real_ratios,outputs_v_high_k,s =20)
ax[2].set_title('Kd = 100*Kr')

for i in range(3):
    ax[i].set_xscale('log')
    ax[i].set_yscale('log')
    ax[i].set_xlim([0.0001,1000])
    ax[i].set_ylim([0.001,10])
    ax[i].set_xlabel('Real ratio input')

fig.set_size_inches(15,5)

## 2-input 2-promoter analysis

Goal:
* find a set 4 transfer functions for 2 promoters (2 for each promoter) that requires ratiometric deconvolution to derive inducer identity

Conditions for success:
* (1) less than 10-fold separation between outputs for 2 different inducers in a given input range for a single promoter
    * this sets up the necessity for some other type of signal processing than looking at a single promoter
    * how to ensure this:
        * make sure 2 transfer function y_max's for a given promoter is not greater than 10
* (2) >10-fold separation between inducers for a range of inputs with ratiometric deconvolution
    * this shows that ratiometric deconvolution is a signal processing solution for the above problem
    


In [ ]:
# Calculates fold difference and ensures its positive by always dividing larger number by smaller number
def pos_fold_calc(a_out,b_out):
    a_out = np.array(a_out)
    b_out = np.array(b_out)
    calc_out = np.zeros(a_out.shape)
    
    # Figure out terms of a that are greater than b
    a_gt_b = np.greater(a_out,b_out)
    
    i = 0
    for i in range(0,calc_out.shape[0]):
        # if a > b for that component. divide a/b; otherwise, divide b/a
        if a_gt_b[i]:
            calc_out[i] = a_out[i] / b_out[i]
        else:
            calc_out[i] = b_out[i] / a_out[i]
        i = i + 1
    return calc_out

In [ ]:
def plt_2p_2in(xs, hill_fn_dict):
    # Promoter 1 response
    # Inducer A
    p1a = hill_fn_dict['P1'][0]
    # Inducer B
    p1b = hill_fn_dict['P1'][1]

    # Promoter 2 response
    # Inducer a
    p2a = hill_fn_dict['P2'][0]
    # Inducer b
    p2b = hill_fn_dict['P2'][1]

    # Calculating real ratios
    p1p2ratios = {'A': [], 'B': []}
    p2p1ratios = {'A': [], 'B': []}

    p1p2ratios['A'] = p1a/p2a
    p2p1ratios['A'] = p2a/p1a
    p1p2ratios['B'] = p1b/p2b
    p2p1ratios['B'] = p2b/p1b
    
    # Calculate results through simulated ratiometric circuit w/ high and low ratio capabilities
    p1p2ratios_low = {'A': [], 'B': []}
    p2p1ratios_low = {'A': [], 'B': []}
    
    p1p2ratios_high = {'A': [], 'B': []}
    p2p1ratios_high = {'A': [], 'B': []}
    
        # Calculate for low ratio circuit
    p1p2ratios_low['A'] = ratio_circ(p1a,p2a)
    p1p2ratios_low['B'] = ratio_circ(p1b,p2b)
    p2p1ratios_low['A'] = ratio_circ(p2a,p1a)
    p2p1ratios_low['B'] = ratio_circ(p2b,p1b)
    
        # Calculate for high ratio circuit    
    p1p2ratios_high['A'] = ratio_circ(p1a,p2a, kd_kr_ratio=10)
    p1p2ratios_high['B'] = ratio_circ(p1b,p2b, kd_kr_ratio=10)
    p2p1ratios_high['A'] = ratio_circ(p2a,p1a, kd_kr_ratio=10)
    p2p1ratios_high['B'] = ratio_circ(p2b,p1b, kd_kr_ratio=10)
    
    # Fold difference calculations
    # Calculate separation with just single promoters
    p1fold = pos_fold_calc(p1a,p1b)
    p2fold = pos_fold_calc(p2a,p2b)
    
    # Calculate fold-separation between different inducers for ratiometric systems
    ratiofold = pos_fold_calc(p1p2ratios['A'],p1p2ratios['B'])
    ratiofold_low = pos_fold_calc(p1p2ratios_low['A'],p1p2ratios_low['B'])
    ratiofold_high = pos_fold_calc(p1p2ratios_high['A'],p1p2ratios_high['B'])
        
    # Plotting
    # Plot it all
    fig, axes = plt.subplots(5,2,sharey=False, sharex = False)

    #p1
    axes[0,0].plot(xs,p1a,color = "#984ea3", linewidth=5)
    axes[0,0].plot(xs,p1b, color="#377eb8",linewidth=5)
    axes[0,0].set_ylim([0.005,5])
    axes[0,0].set_title('P1')

    #p2
    axes[0,1].plot(xs,p2a,color = "#984ea3", linewidth=5)
    axes[0,1].plot(xs,p2b,color="#377eb8",linewidth=5)
    axes[0,1].set_ylim([0.005,5])
    axes[0,1].set_title('P2')

    #p1/p2
    axes[1,0].plot(xs,p1p2ratios['A'],color = "#984ea3", linewidth=5)
    axes[1,0].plot(xs,p1p2ratios['B'],color="#377eb8",linewidth=5)
    axes[1,0].set_ylim([0.01,100])
    axes[1,0].set_ylabel('Real Ratio')
    axes[1,0].set_title('P1/P2')

    #p2/p1
    axes[1,1].plot(xs,p2p1ratios['A'],color = "#984ea3", linewidth=5)
    axes[1,1].plot(xs,p2p1ratios['B'],color="#377eb8",linewidth=5)
    axes[1,1].set_ylim([0.01,100])
    axes[1,1].set_ylabel('Real Ratio')
    axes[1,1].set_title('P2/P1')
    
    #p1/p2 low ratio
    axes[2,0].plot(xs,p1p2ratios_low['A'],color = "#984ea3", linewidth=5)
    axes[2,0].plot(xs,p1p2ratios_low['B'],color="#377eb8",linewidth=5)
    axes[2,0].set_ylim([0.01,100])
    axes[2,0].set_ylabel('Ratiometric circuit output')
    axes[2,0].set_title('P2/P1, low ratio')
    
    #p2/p1 low ratio
    
    axes[2,1].plot(xs,p2p1ratios_low['A'],color = "#984ea3", linewidth=5)
    axes[2,1].plot(xs,p2p1ratios_low['B'],color="#377eb8",linewidth=5)
    axes[2,1].set_ylim([0.01,100])
    axes[2,1].set_ylabel('Ratiometric circuit output')
    axes[2,1].set_title('P2/P1, low ratio')
    
    #p1/p2 high ratio
    
    axes[3,0].plot(xs,p1p2ratios_high['A'],color = "#984ea3", linewidth=5)
    axes[3,0].plot(xs,p1p2ratios_high['B'],color="#377eb8",linewidth=5)
    axes[3,0].set_ylim([0.01,100])
    axes[3,0].set_ylabel('Ratiometric circuit output')
    axes[3,0].set_title('P2/P1, high ratio')
    
    #p2/p1 high ratio
    
    axes[3,1].plot(xs,p2p1ratios_low['A'],color = "#984ea3", linewidth=5)
    axes[3,1].plot(xs,p2p1ratios_low['B'],color="#377eb8",linewidth=5)
    axes[3,1].set_ylim([0.01,100])
    axes[3,1].set_ylabel('Ratiometric circuit output')
    axes[3,1].set_title('P2/P1, high ratio')
    
    # fold differences for p1 only, p2 only, and real ratios
    axes[4,0].plot(xs,p1fold, color='#984ea3', linewidth = 5)
    axes[4,0].plot(xs,p2fold, color='#377eb8', linewidth = 5)
    axes[4,0].plot(xs,ratiofold, color='#000000', linewidth = 5)
    axes[4,0].legend(['P1 only', 'P2 only', 'Ratio'])
    axes[4,0].set_yscale('log')
    axes[4,0].set_xscale('log')
    axes[4,0].set_xlabel('Input')
    axes[4,0].set_ylabel('Fold-difference')
    axes[4,0].set_title('Fold separation w/ ideal ratio')
    
    # fold differences for p1 only, p2 only, and ratios through circuit
    axes[4,1].plot(xs,p1fold, color='#984ea3', linewidth = 5)
    axes[4,1].plot(xs,p2fold, color='#377eb8', linewidth = 5)
    axes[4,1].plot(xs,ratiofold_low, color='#000000', linewidth = 5)
    axes[4,1].plot(xs,ratiofold_high, color='#C4C4C4', linewidth = 5)
    axes[4,1].legend(['P1 only', 'P2 only', 'Ratio circ (low)', 'Ratio circ (high)'])
    axes[4,1].set_yscale('log')
    axes[4,1].set_xscale('log')
    axes[4,1].set_xlabel('Input')
    axes[4,1].set_ylabel('Fold-difference')
    axes[4,1].set_title('Fold separation w/ real circuit')
    

    for i in [0,1,2,3]:
        for j in [0,1]:
            axes[i,j].set_xscale('log')
            axes[i,j].set_yscale('log')
            axes[i,j].set_xlabel('Input')
            if i != 1:
                axes[i,j].set_ylabel('Output')
            axes[i,j].legend(['A','B'])

    sns.despine()

    fig.set_size_inches(15,25)
    fig.tight_layout()
    fig.subplots_adjust(hspace = 1)

In [ ]:
# Inducer A, then Inducer B for each promoter
xs = np.logspace(-2,1,100)
hill_fx = {
    'P1':[hill(xs, y_max = 0.1), hill(xs, kd = 5, y_max = 0.1)],
    'P2':[hill(xs, kd = 0.2, n = 4), hill(xs, kd = 2, y_max = 0.05)]
}

plt_2p_2in(xs,hill_fx)

### Results
* Resolving: at least 10-fold separation from inputs of $10^{-1}$ and up
* this does NOT yield new information since we could just look at P2 and see that we get more than 10-fold separation if input is more than $10^{-1}$
* Issue:
    * P2 transfer functions have 10-fold separation from inputs of $2 * 10^{-1}$ and up

In [ ]:
hill_fx = {
    'P1':[hill(xs, y_max = 0.05,kd = 5), hill(xs,  y_max = 0.1, kd=1)],
    'P2':[hill(xs, kd = 1, y_max = 0.2), hill(xs, kd = 5, y_max = 0.1)]
}

plt_2p_2in(xs,hill_fx)

### Results
* ratiometric circuit results in better separation than either fold-separation alone

These transfer functions are nearly identical to CinR and LasR promoters being induced by each others ligands

<img src="cin_las_crosstalk.png">

## CinR / LasRQ simulation
* attempt to simulate the transfer functions for CinR / LasRQ promoters from Adam's data
* P1 = CinR promoter
* P2 = LasRQ promoter
* Inducer A = CinR ligand
* Inducer B = LasRQ ligand

In [ ]:
# Inducer A, then Inducer B for each promoter
xs = np.logspace(-7,0,100)
hill_fx = {
    'P1':[hill(xs, y_min = 0.02, y_max = 2, kd = 2*10**(-5), n = 2), 
          hill(xs, y_min = 0.01, y_max = 1, kd = 4*10**(-2), n = 2)],
    'P2':[hill(xs, y_min = 0.03, y_max = 1, kd = 9*10**(-2), n = 1), 
          hill(xs, y_min = 0.03, y_max = 2, kd = 7*10**(-4), n = 1)]
}

plt_2p_2in(xs,hill_fx)

## Results
* Looks like theres a really good range here from ~ $10^{-5}$ -> $10^{-1}$ where the ratio gives a distinct advantage over P1 and P2 alone
* might not be as big of an advantage if ratio can't be far above 1

# Resolving 2 signals w/ 1 cross-talking signal

In [ ]:
# Inducer A, then Inducer B for each promoter
xs = np.logspace(-7,0,100)
hill_fx = {
    'P1':[hill(xs, y_min = 0.02, y_max = 2, kd = 2*10**(-3), n = 2), 
          hill(xs, y_min = 0.02, y_max = 0.02, kd = 4*10**(-2), n = 2)],
    'P2':[hill(xs, y_min = 0.03, y_max = 0.5, kd = 9*10**(-2), n = 1), 
          hill(xs, y_min = 0.03, y_max = 1, kd = 7*10**(-4), n = 1)]
}

plt_2p_2in(xs,hill_fx)

## Results
* could work but if only one input is present at a time (A or B)
* doesn't work to deconvolve signals if both are present at the same time

## Ratiometric comparison with different Kd values and y_max (3 inputs)

Situation that Adam said was most common for the AHLs

In [ ]:
# Set output values for all inducers

# x values for all inducers
xs = np.logspace(-2,1,100)

# inducer output values for each sensor, indcuer name key-valued lists
outs_s1 = {'A': [], 'B': [], 'C': []}
outs_s2 = {'A': [], 'B': [], 'C': []}

# set sensor 1 values
# A is default
outs_s1['A'] = hill(xs)
# B has shifted KD and lower ymax
outs_s1['B'] = hill(xs, kd = 5, y_max = 0.1)
# C has same KD as B, but lower ymax
outs_s1['C'] = hill(xs, kd = 5, y_max = 0.05)

# set sensor 2 values
# A is higher KD and shifted down
outs_s2['A'] = hill(xs, kd = 2, y_max = 0.05)
# B is default
outs_s2['B'] = hill(xs)
# C has same KD as A, but lower ymax
outs_s2['C'] = hill(xs, kd = 2, y_max = 0.02)

Inducer transfer functions

In [ ]:
# Plot the inducer transfer functions
fig, axes = plt.subplots(1,2)

axes[0].plot(xs,outs_s1['A'],color = "#377eb8", linewidth=5)
axes[0].plot(xs,outs_s1['B'],color="#e41a1c",linewidth=5)
axes[0].plot(xs,outs_s1['C'],color="#4daf4a",linewidth=5)

axes[0].set_xscale('log')
axes[0].set_yscale('log')

axes[1].plot(xs,outs_s2['A'],color = "#377eb8", linewidth=5)
axes[1].plot(xs,outs_s2['B'],color="#e41a1c",linewidth=5)
axes[1].plot(xs,outs_s2['C'],color="#4daf4a",linewidth=5)

    
axes[1].set_xscale('log')
axes[1].set_yscale('log')

sns.despine()

fig.set_size_inches(12,5)
plt.show()

## Ratiometric comparison

In [ ]:
# Calculate ratios over range of input inducer concentrations; Calculate sensor1/sensor2

ratios = {'A': [], 'B': [], 'C': []}

for ind_name, ratio_vals in ratios.items():
    ratios[ind_name] = np.divide(outs_s1[ind_name], outs_s2[ind_name])

In [ ]:
# Plot the ratios for each

fig, ax = plt.subplots()

ax.plot(xs,ratios['A'],color = "#377eb8", linewidth=5)
ax.plot(xs,ratios['B'],color="#e41a1c",linewidth=5)
ax.plot(xs,ratios['C'],color="#4daf4a",linewidth=5)
    
ax.set_xscale('log')
ax.set_yscale('log')
sns.despine()

fig.set_size_inches(6,6.5)

## 3-inputs, one differentiated

Goal:
* Simulate a case where 3 signals are parsed to detect only 1 metal

In [ ]:
# Set output values for all inducers

# x values for all inducers
xs = np.logspace(-2,1,100)

# inducer output values for each sensor, indcuer name key-valued lists
outs_s1 = {'A': [], 'B': [], 'C': []}
outs_s2 = {'A': [], 'B': [], 'C': []}

# set sensor 1 values
# A is default
outs_s1['A'] = hill(xs, kd = 0.5, y_max = 1)
# B has shifted KD and lower ymax
outs_s1['B'] = hill(xs, kd = 0.5, y_max = 0.5)
# C has same KD as B, but lower ymax
outs_s1['C'] = hill(xs, kd = 0.5, y_max = 0.01)

# set sensor 2 values
# A is higher KD and shifted down
outs_s2['A'] = hill(xs, kd = 5, y_max = 0.5)
# B is default
outs_s2['B'] = hill(xs, kd = 1, y_max = 0.01)
# C has same KD as A, but lower ymax
outs_s2['C'] = hill(xs, kd = 1, y_max = 1)

In [ ]:
# Plot the inducer transfer functions
fig, axes = plt.subplots(1,2)

axes[0].plot(xs,outs_s1['A'],color = "#377eb8", linewidth=5)
axes[0].plot(xs,outs_s1['B'],color="#e41a1c",linewidth=5)
axes[0].plot(xs,outs_s1['C'],color="#4daf4a",linewidth=5)
axes[0].legend(['Cd', 'Pb', 'As'])

axes[0].set_xscale('log')
axes[0].set_yscale('log')

axes[1].plot(xs,outs_s2['A'],color = "#377eb8", linewidth=5)
axes[1].plot(xs,outs_s2['B'],color="#e41a1c",linewidth=5)
axes[1].plot(xs,outs_s2['C'],color="#4daf4a",linewidth=5)

    
axes[1].set_xscale('log')
axes[1].set_yscale('log')

sns.despine()

fig.set_size_inches(12,5)
plt.show()

In [ ]:
# Calculate ratios over range of input inducer concentrations; Calculate sensor1/sensor2

ratios = {'A': [], 'B': [], 'C': []}

for ind_name, ratio_vals in ratios.items():
    ratios[ind_name] = np.divide(outs_s1[ind_name], outs_s2[ind_name])
    
# Plot the ratios for each

fig, ax = plt.subplots()

ax.plot(xs,ratios['A'],color = "#377eb8", linewidth=5)
ax.plot(xs,ratios['B'],color="#e41a1c",linewidth=5)
ax.plot(xs,ratios['C'],color="#4daf4a",linewidth=5)
ax.legend(['Cd', 'Pb', 'As'])

ax.set_xscale('log')
ax.set_yscale('log')
sns.despine()

fig.set_size_inches(6,6.5)

## Ratiometric comparison w/ 4 signals

* 2 sensors, 4 inputs

In [ ]:
# Set output values for all inducers

# x values for all inducers
xs = np.logspace(-2,1,100)

# inducer output values for each sensor, indcuer name key-valued lists
outs_s1 = {'A': [], 'B': [], 'C': [],'D': []}
outs_s2 = {'A': [], 'B': [], 'C': [],'D': []}
outs_s3 = {'A': [], 'B': [], 'C': [],'D': []}

# set sensor 1 values
# A is default
outs_s1['A'] = hill(xs)
# B has shifted KD and lower ymax
outs_s1['B'] = hill(xs, kd = 5, y_max = 0.1)
# C has same KD as B, but lower ymax
outs_s1['C'] = hill(xs, kd = 5, y_max = 0.05)
outs_s1['D'] = hill(xs, kd = 0.1, y_max = 0.5)

# set sensor 2 values
# A is higher KD and shifted down
outs_s2['A'] = hill(xs, kd = 1, y_max = 0.05)
# B is default
outs_s2['B'] = hill(xs)
# C has same KD as A, but lower ymax
outs_s2['C'] = hill(xs, kd = 2, y_max = 0.02)
outs_s2['D'] = hill(xs, kd = 8, y_max = 0.1)

In [ ]:
fig, ax = plt.subplots()

ax.plot(xs,outs_s1['A'],color = "#377eb8", linewidth=5)
ax.plot(xs,outs_s1['B'],color="#e41a1c",linewidth=5)
ax.plot(xs,outs_s1['C'],color="#4daf4a",linewidth=5)
ax.plot(xs,outs_s1['D'],color="#984ea3",linewidth=5)

    
ax.set_xscale('log')
ax.set_yscale('log')
sns.despine()

fig.set_size_inches(6,5)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(xs,outs_s2['A'],color = "#377eb8", linewidth=5)
ax.plot(xs,outs_s2['B'],color="#e41a1c",linewidth=5)
ax.plot(xs,outs_s2['C'],color="#4daf4a",linewidth=5)
ax.plot(xs,outs_s2['D'],color="#984ea3",linewidth=5)

    
ax.set_xscale('log')
ax.set_yscale('log')
sns.despine()

fig.set_size_inches(6,5)
plt.show()

In [ ]:
# Calculate ratios over range of input inducer concentrations; Calculate sensor1/sensor2

ratios = {'A': [], 'B': [], 'C': [], 'D':[]}

for ind_name, ratio_vals in ratios.items():
    ratios[ind_name] = np.divide(outs_s1[ind_name], outs_s2[ind_name])

# Plot the ratios for each

fig, ax = plt.subplots()

ax.plot(xs,ratios['A'],color = "#377eb8", linewidth=5)
ax.plot(xs,ratios['B'],color="#e41a1c",linewidth=5)
ax.plot(xs,ratios['C'],color="#4daf4a",linewidth=5)
ax.plot(xs,ratios['D'],color="#984ea3",linewidth=5)
    
ax.set_xscale('log')
ax.set_yscale('log')
sns.despine()

fig.set_size_inches(6,6.5)

## Low-fold change ratiometric
* use three sensors to differentiate three low-fold change 

In [ ]:
# Set output values for all inducers

# x values for all inducers
xs = np.logspace(-2,1,100)

# inducer output values for each sensor, indcuer name key-valued lists
outs_s1 = {'A': [], 'B': [], 'C': [],'D': []}
outs_s2 = {'A': [], 'B': [], 'C': [],'D': []}
outs_s3 = {'A': [], 'B': [], 'C': [],'D': []}

# set sensor 1 values
# A is default
outs_s1['A'] = hill(xs)
# B has shifted KD and lower ymax
outs_s1['B'] = hill(xs, kd = 5, y_max = 0.1)
# C has same KD as B, but lower ymax
outs_s1['C'] = hill(xs, kd = 5, y_max = 0.05)
outs_s1['D'] = hill(xs, kd = 0.1, y_max = 0.5)

# set sensor 2 values
# A is higher KD and shifted down
outs_s2['A'] = hill(xs, kd = 1, y_max = 0.05)
# B is default
outs_s2['B'] = hill(xs)
# C has same KD as A, but lower ymax
outs_s2['C'] = hill(xs, kd = 2, y_max = 0.02)
outs_s2['D'] = hill(xs, kd = 8, y_max = 0.1)

# A is higher KD and shifted down
outs_s2['A'] = hill(xs, kd = 1, y_max = 0.05)
# B is default
outs_s2['B'] = hill(xs)
# C has same KD as A, but lower ymax
outs_s2['C'] = hill(xs, kd = 2, y_max = 0.02)
outs_s2['D'] = hill(xs, kd = 8, y_max = 0.1)